In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
%%time

import malaya
malaya

CPU times: user 3.02 s, sys: 2.64 s, total: 5.66 s
Wall time: 3.36 s


/home/ubuntu/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/dev/malaya/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


<module 'malaya' from '/home/ubuntu/dev/malaya/malaya/__init__.py'>

In [3]:
sastrawi = malaya.stem.sastrawi()

In [4]:
# !wget https://github.com/huseinzol05/malay-dataset/blob/master/normalization/stemmer/stem.zip?raw=true
# !wget https://f000.backblazeb2.com/file/malay-dataset/wiki-stem.json
# !unzip stem.zip?raw=true

In [5]:
with open('/home/ubuntu/malaya/stemmer-data-v5.txt') as fopen:
    stemming = fopen.read().split('\n')
stemming[:5]

['spider\tspider',
 'transeksual\ttranseksual',
 'khusus\tkhusus',
 'kriminologi\tkriminologi',
 'negeripingat\tnegeripingat']

In [6]:
left, right = [], []

for s in stemming:
    l, r = s.split('\t')
    left.append(l)
    right.append(r)
    
len(left)

41528

In [7]:
import json

with open('/home/ubuntu/malaya/news-stem.json') as fopen:
    news = json.load(fopen)
    
for s in news:
    left.append(s[0])
    right.append(s[1])

In [8]:
with open('/home/ubuntu/malaya/wiki-stem.json') as fopen:
    wiki = json.load(fopen)
    
for s in wiki:
    left.append(s[0])
    right.append(s[1])

In [9]:
len(left), len(right)

(1832338, 1832338)

In [10]:
left[1], right[0]

('transeksual', 'spider')

In [11]:
sastrawi.stem(f'{left[1]}lah')

'transeksual'

In [12]:
stem = sastrawi.stem('sakitnya')
stem

'sakit'

In [13]:
hujung = {
    'kan': ['kann', 'kkan', 'kaann', 'kn', 'kkn'],
    'lah': ['lh', 'lhh', 'lhaa', 'lha', 'laah', 'laahh', 'ler'],
    'nya': ['nye', 'ny', 'nyee', 'nyye', 'nyo', 'nyoo'],
}

In [14]:
import re
import random

vowels = 'aeiou'
consonants = 'bcdfghjklmnpqrstvwxyz'

In [15]:
def augment_hubung(word, p_remove_vowels = 0.5, p_duplicate = 0.7, min_duplicate = 2, max_duplicate = 3):
    word = ['' if c in vowels and random.random() > p_remove_vowels else c for c in word]
    word = list(filter(None, word))
    for i in range(len(word)):
        w = word[i]
        if random.random() > p_duplicate:
            duplicate = random.randint(min_duplicate, max_duplicate)
            word[i] = w * duplicate
    return ''.join(word)

In [16]:
malaya.augmentation.vowel_alternate('sakit')

'skt'

In [17]:
augment_hubung('ter')

'tr'

In [18]:
string = 'tersakitnya'
stemmed = sastrawi.stem(string)
index = [(m.start(0), m.end(0)) for m in re.finditer(stemmed, string, flags=re.IGNORECASE)][0]
permulaan = string[:index[0]]
hujungan = string[index[1]:]
permulaan, hujungan

('ter', 'nya')

In [19]:
awalan = ['pe', 'pem', 'pen', 'peng', 'pel', 'ke', 'juru', 'pra', 'dwi',
         'me', 'memper', 'ber', 'ter', 'di', 'diper', 'ter', 'te', 'se',
         'mem', 'diper']

akhiran = ['lah', 'an', 'kan', 'nya', 'wan', 'wati']

In [20]:
random.choice(hujung.get(hujungan, [hujungan]))

'ny'

In [21]:
sastrawi.stem('memperhambakan')

'hamba'

In [22]:
set_left = set(left)

In [23]:
from tqdm import tqdm

In [25]:
new_left, new_right = [], []
for w in tqdm(malaya.text.bahasa.dbp.words | malaya.text.bahasa.cambridge_words.words | malaya.text.bahasa.kamus_dewan.words | malaya.text.bahasa.words.words):
    if w in set_left:
        continue
    s = sastrawi.stem(w)
    new_left.append(w)
    new_right.append(s)
    set_left.add(w)
    
left.extend(new_left)
right.extend(new_right)

100%|██████████████████████████████████████████████████████████████████████████████████| 63838/63838 [00:04<00:00, 15738.89it/s]


In [26]:
len(left), len(right)

(1855109, 1855109)

In [27]:
def augment(word, p_augment_word = 0.7):
    awal = random.choice(awalan)
    akhir = random.choice(akhiran)
    joined = awal + word + akhir
    stemmed = sastrawi.stem(joined)
    if stemmed == joined:
        return word, word
    
    index = [(m.start(0), m.end(0)) for m in re.finditer(stemmed, joined, flags=re.IGNORECASE)][0]
    permulaan = joined[:index[0]]
    hujungan = joined[index[1]:]
    
    if not len(permulaan) and not len(hujungan):
        return False, left, right
    
    print(joined, stemmed, permulaan, hujungan)
    
    augmented_permulaan = augment_hubung(permulaan)
    if len(augmented_permulaan) > 1:
        permulaan = augmented_permulaan
    
    augmented_hujungan = augment_hubung(hujungan)
    if len(augmented_hujungan) > 1:
        hujungan = augmented_hujungan
        
    if random.random() > p_augment_word:
        word = malaya.augmentation.vowel_alternate(word)
    
    return True, permulaan + word + hujungan, word

def augment_left_right(left, right, p_augment_word = 0.7):
    index = [(m.start(0), m.end(0)) for m in re.finditer(right, left, flags=re.IGNORECASE)]
    permulaan = left[:index[0][0]]
    hujungan = left[index[-1][1]:]

    not_right = left[index[0][0]:index[-1][1]]
    # print(permulaan, hujungan, not_right)
    
    if not len(permulaan) and not len(hujungan):
        return False, left, right
    
    augmented_permulaan = augment_hubung(permulaan)
    if len(augmented_permulaan) > 1:
        permulaan = augmented_permulaan
    
    augmented_hujungan = augment_hubung(hujungan)
    if len(augmented_hujungan) > 1:
        hujungan = augmented_hujungan
    
    if random.random() > p_augment_word and not_right == right:
        right = malaya.augmentation.vowel_alternate(right)
        selected = right
    else:
        selected = not_right
    
    joined = permulaan + selected + hujungan
    joined = re.sub(r'^-+|-+$', '-', joined).strip()
    if joined == left:
        result = False
    else:
        result = True
    
    return result, joined, right

In [28]:
not_same = []
for i in range(len(left)):
    left_ = left[i].lower()
    left_ = re.sub(r'^-+|-+$', '-', left_).strip().replace('"', ' ').replace('\'', ' ')
    left_ = re.sub(r'[ ]+', ' ', left_).strip()
    right_ = right[i].lower()
    
    if left_ != right_ and left[i].lower() != right[i].lower():
        left_ = re.sub(r'^-+|-+$', '-', left[i]).strip().replace('"', ' ').replace('\'', ' ')
        left_ = re.sub(r'[ ]+', ' ', left_).strip()
        not_same.append((left_, right[i]))
        
len(not_same), len(not_same) / len(left)

(189778, 0.1023001882908228)

In [29]:
not_same[-1000:]

[('seikhwan', 'ikhwan'),
 ('memangkalkan', 'pangkal'),
 ('menimbau', 'timbau'),
 ('kocekan', 'kocek'),
 ('memetiakan', 'petia'),
 ('menyambit', 'sambit'),
 ('diperdomi', 'dom'),
 ('menyimpirkan', 'simpir'),
 ('merampai', 'rampai'),
 ('kotorkan', 'kotor'),
 ('berkecapi', 'kecap'),
 ('perepangan', 'repang'),
 ('keterdorongan', 'dorong'),
 ('berkersik-kersik', 'kersik'),
 ('campak-mencampak', 'campak'),
 ('olah-olah', 'olah'),
 ('menangkaskan', 'tangkas'),
 ('perekayasaan', 'rekayasa'),
 ('rintangkan', 'rintang'),
 ('berlarik', 'larik'),
 ('petandang', 'tandang'),
 ('menjelimpat', 'jelimpat'),
 ('melengangkan', 'lengang'),
 ('meranjah', 'ranjah'),
 ('penyemarakan', 'semara'),
 ('terigau-igau', 'igau'),
 ('kedemi', 'demi'),
 ('lipatnya', 'lipat'),
 ('menghitung-hitung', 'hitung'),
 ('menjelepok', 'jelepok'),
 ('berfirasat', 'firasat'),
 ('keperang-perangan', 'perang'),
 ('terpercik-percik', 'percik'),
 ('kericik', 'ricik'),
 ('mengobor', 'obor'),
 ('menumpah-numpahkan', 'tumpah'),
 ('menge

In [39]:
augment_left_right('mengenjak-enjak', 'enjak')

(True, 'mmngenjak-enjak', 'enjak')